In [ ]:
# STEP 1
%pip install pandas numpy
%pip install scipy
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# STEP 2
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

In [ ]:
# STEP 3
base_path = "/content/drive/MyDrive/newdataset"

gt_path = f"{base_path}/groundtruth_2013-01-10.csv"
odom_path = f"{base_path}/odometry_mu_100hz.csv"
lidar_path = f"{base_path}/lidar.csv"
output_path = f"{base_path}/aligned_scans_final1.csv"

In [ ]:
# STEP 4
gt = pd.read_csv(gt_path, header=None)[[0, 1, 2, 3, 4, 5, 6]]
gt.columns = ['Time', 'GT_X', 'GT_Y', 'GT_Z', 'GT_Roll', 'GT_Pitch', 'GT_Yaw']
gt = gt.sort_values('Time')

odom = pd.read_csv(odom_path, header=None)[[0, 1, 2, 3, 4, 5, 6]]
odom.columns = ['Time', 'Odom_X', 'Odom_Y', 'Odom_Z', 'Odom_Roll', 'Odom_Pitch', 'Odom_Yaw']
odom = odom.sort_values('Time')

lidar = pd.read_csv(lidar_path)[['utime', 'x', 'y']]
lidar.columns = ['Time', 'LiDAR_X', 'LiDAR_Y']
lidar = lidar.sort_values('Time').reset_index(drop=True)

In [ ]:
# STEP 5
min_gt_time, max_gt_time = gt['Time'].min(), gt['Time'].max()
odom = odom[(odom['Time'] >= min_gt_time) & (odom['Time'] <= max_gt_time)]

interp_axes = ['Odom_X', 'Odom_Y', 'Odom_Z', 'Odom_Roll', 'Odom_Pitch', 'Odom_Yaw']
odom_interp = {
    axis: interp1d(odom['Time'], odom[axis], kind='linear', fill_value="extrapolate")
    for axis in interp_axes
}

for axis in interp_axes:
    gt[axis] = odom_interp[axis](gt['Time'])

In [ ]:
# STEP 6
lidar_scans = {}
for t, group in lidar.groupby('Time'):
    if len(group) == 1081:
        lidar_scans[t] = group[['LiDAR_X', 'LiDAR_Y']].to_numpy()

lidar_times = np.array(list(lidar_scans.keys()))
used_lidar_times = set()

In [ ]:
# STEP 7
output_rows = []

for _, row in gt.iterrows():
    gt_time = row['Time']

    valid_times = [t for t in lidar_times if t >= min_gt_time and t not in used_lidar_times]
    if not valid_times:
        continue

    closest_t = min(valid_times, key=lambda t: abs(t - gt_time))

    if abs(closest_t - gt_time) > 1e6:
        continue

    scan = lidar_scans.get(closest_t)
    if scan is None:
        continue

    used_lidar_times.add(closest_t)

    lidar_x = scan[:, 0].tolist()
    lidar_y = scan[:, 1].tolist()

    output_rows.append({
        'Time': gt_time,
        'GT_X': row['GT_X'], 'GT_Y': row['GT_Y'], 'GT_Z': row['GT_Z'],
        'GT_Roll': row['GT_Roll'], 'GT_Pitch': row['GT_Pitch'], 'GT_Yaw': row['GT_Yaw'],
        'Odom_X': row['Odom_X'], 'Odom_Y': row['Odom_Y'], 'Odom_Z': row['Odom_Z'],
        'Odom_Roll': row['Odom_Roll'], 'Odom_Pitch': row['Odom_Pitch'], 'Odom_Yaw': row['Odom_Yaw'],
        'LiDAR_X': lidar_x,
        'LiDAR_Y': lidar_y
    })

In [ ]:
# STEP 8
final_df = pd.DataFrame(output_rows)
final_df.to_csv(output_path, index=False)

print(f"✅ Done! Final output saved to: {output_path}")

✅ Done! Final output saved to: /content/drive/MyDrive/newdataset/aligned_scans_vectorized_with_angles.csv
